In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('flights.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   flight_id            386 non-null    int64  
 1   flight_no            386 non-null    object 
 2   scheduled_departure  386 non-null    object 
 3   scheduled_arrival    386 non-null    object 
 4   actual_departure     386 non-null    object 
 5   actual_arrival       386 non-null    object 
 6   scheduled_duration   386 non-null    float64
 7   actual_duration      386 non-null    float64
 8   departure_delay      386 non-null    float64
 9   arrival_delay        386 non-null    float64
 10  departure_airport    386 non-null    object 
 11  departure_city       386 non-null    object 
 12  arrival_airport      386 non-null    object 
 13  arrival_city         386 non-null    object 
 14  status               386 non-null    object 
 15  aircraft_code        386 non-null    obj

# Описание данных

### С помощью запроса мы получили данные о полётах в зимние месяцы:
flight_id - ID полёта<br />
flight_no - No рейса<br />
scheduled_departure - Запланированное время отправления<br />
scheduled_arrival - Запланированное время прибытия<br />
actual_departure - Действительное время отправления<br />
actual_arrival - Действительное время прибытия<br />
scheduled_duration - Задержка запланированного времени<br />
actual_duration - Действительная задержка<br />
departure_delay - Задержка отправления<br />
arrival_delay - Задержка прибытия<br />
departure_airport - Аэропорт вылета<br />
departure_city - Город вылета<br />
arrival_airport - Аэропорт прибытия<br />
arrival_city - Город прибытия<br />
status - Статус<br />
aircraft_code - Код самолёта<br />
model - Модель самолёта<br />
range - Дальность полёта самолёта<br />
seats_amount - Кол-во мест в самолёте<br />
tickets_amount - Кол-во проданных билетов<br />
paid_total - Общая уплаченная сумма

In [2]:
df.sample(5)

,flight_id,flight_no,scheduled_departure,scheduled_arrival,actual_departure,actual_arrival,scheduled_duration,actual_duration,departure_delay,arrival_delay,...,departure_city,arrival_airport,arrival_city,status,aircraft_code,model,range,seats_amount,tickets_amount,paid_total
231,136757,PG0480,2017-01-27T09:25:00Z,2017-01-27T10:15:00Z,2017-01-27T09:30:00Z,2017-01-27T10:19:00Z,50.0,49.0,5.0,4.0,...,Anapa,EGO,Belgorod,Arrived,SU9,Sukhoi Superjet-100,3000,97,97,765300.0
120,136253,PG0252,2016-12-24T10:05:00Z,2016-12-24T11:45:00Z,2016-12-24T10:08:00Z,2016-12-24T11:46:00Z,100.0,98.0,3.0,1.0,...,Anapa,SVO,Moscow,Arrived,733,Boeing 737-300,4200,130,93,1407800.0
47,41499,PG0251,2017-02-27T15:05:00Z,2017-02-27T16:45:00Z,2017-02-27T15:13:00Z,2017-02-27T16:52:00Z,100.0,99.0,8.0,7.0,...,Moscow,AAQ,Anapa,Arrived,733,Boeing 737-300,4200,130,117,1703000.0
76,41594,PG0251,2017-01-24T15:05:00Z,2017-01-24T16:45:00Z,2017-01-24T15:08:00Z,2017-01-24T16:49:00Z,100.0,101.0,3.0,4.0,...,Moscow,AAQ,Anapa,Arrived,733,Boeing 737-300,4200,130,117,1677400.0
25,41366,PG0251,2016-12-23T15:05:00Z,2016-12-23T16:45:00Z,2016-12-23T15:13:00Z,2016-12-23T16:54:00Z,100.0,101.0,8.0,9.0,...,Moscow,AAQ,Anapa,Arrived,733,Boeing 737-300,4200,130,129,1873800.0


In [3]:
df['aircraft_code'].value_counts()

733    206
SU9    180
Name: aircraft_code, dtype: int64

In [4]:
df['arrival_city'].value_counts()

Anapa           193
Moscow           90
Belgorod         90
Novokuznetsk     13
Name: arrival_city, dtype: int64

Расстояния:<br />
    до Белгорода ~ 650 км. (50 мин.)<br />
    до Москвы ~ 1200 км. (100 мин.)<br />
    до Новокузнецка ~ 3600 км. (300 мин)

# Преобразуем данные

In [5]:
# Добавим информацию о стоимости полёта, получим прибыль

# расход топлива, кг/час * 40 рублей за кг = руб/час
fuel_consumption = {
    '733': 2600 * 40,
    'SU9': 1700 * 40,
}

df['flight_cost'] = df.apply(lambda x: round(fuel_consumption[x['aircraft_code']] * x['actual_duration'] / 60), axis = 1)

df['paid_total'].fillna(0, inplace=True)

df['revenue'] = df['paid_total'] - df['flight_cost']

In [6]:
# Добавим информацию о дальности полёта

cities = {
    'Moscow': 1200,
    'Belgorod': 650,
    'Novokuznetsk': 3600,
}

def actual_city(row):
    if row['departure_city'] in cities.keys() :
        return cities[row['departure_city']]
    if row['arrival_city'] in cities.keys() :
        return cities[row['arrival_city']]
    raise 'Oops!..'

df['actual_range'] = df.apply (lambda row: actual_city(row), axis=1)

df['actual_range'].value_counts()

1200    180
650     180
3600     26
Name: actual_range, dtype: int64

In [7]:
# Предположим, что рейс совершит другой самолёт

# поменяем местами расход топлива
fuel_consumption = {
    'SU9': 2600 * 40, # was 733
    '733': 1700 * 40, # was SU9
}

# посчитаем предполагаемую стоимость рейса
df['possible_flight_cost'] = df.apply(lambda x: round(fuel_consumption[x['aircraft_code']] * x['actual_duration'] / 60), axis = 1)

# предполагаемая прибыль
df['possible_revenue'] = df['paid_total'] - df['possible_flight_cost']

# поменяем местами максимальную дальность полёта
df['possible_range'] = df.apply(lambda x: 3000 if x['range'] == 4200 else 4200 , axis = 1)

# перемена модели самолёта возможна, если у новой модели сопоставимая дальность
df['revenue_possibility'] = df.apply(lambda x: x['possible_range'] > x['actual_range'] , axis = 1)
# В большинстве случаев это возможно
# True     360
# False     26

# соответственно, возможная прибыль составит
df['possible_revenue'] = df.apply(lambda x: x['possible_revenue'] if x['revenue_possibility'] else 0, axis = 1)

# если возможная прибылть больше действительной, смена модели самолёта обоснована
df['change_flight_reasonableness'] = df.apply(lambda x: x['possible_revenue'] > x['revenue'], axis = 1)

# выгода от смены самолёта составит
df['change_flight_revenue_delta'] = df.apply(
    lambda x: 
        x['possible_revenue'] - x['revenue'] 
            if (x['change_flight_reasonableness'] and x['revenue_possibility']) 
        else -1,
    axis = 1)

In [8]:
# Процент заполненности самолёта пассажирами
df['fullness'] = df.apply(lambda x: x['tickets_amount']*100/x['seats_amount'], axis = 1)

In [9]:
# посчитаем средние показатели для каждого рейса
columns = [
    'flight_no',
    'aircraft_code',
    'arrival_city',
    'fullness',
    'revenue',
    'change_flight_revenue_delta'
]

flights = list(df['flight_no'].value_counts().index)
mean_values = []
for f in flights:
    values = [f]
    for i in columns[1:]:
        values.append(
            df[df['flight_no'] == f][i].iloc[0]
            if i in ['aircraft_code', 'arrival_city'] else
            round(df[df['flight_no'] == f][i].mean())
        )
    mean_values.append(values)

pd.DataFrame(mean_values, columns=columns).sort_values('revenue', ascending=False)

,flight_no,aircraft_code,arrival_city,fullness,revenue,change_flight_revenue_delta
0,PG0251,733,Anapa,89,1502473,60033
1,PG0252,733,Moscow,87,1473622,59980
3,PG0481,SU9,Anapa,95,673849,-1
2,PG0480,SU9,Belgorod,93,655717,-1
5,PG0195,733,Anapa,0,-525467,-1
4,PG0194,733,Novokuznetsk,0,-530133,-1


In [10]:
# От каких рейсов можно отказаться
suggested_flights = {}

In [11]:
# посмотрим самые неприбыльные рейсы
flights_top = df[df['revenue'] > 0][['flight_id'] + columns].sort_values('revenue', ascending=True)[:10]

for x in list(flights_top['flight_id']):
    suggested_flights[x] = 1 if x not in suggested_flights else suggested_flights[x] + 1

suggested_flights_list = flights_top[:3]

display(suggested_flights_list)
display(flights_top)


,flight_id,flight_no,aircraft_code,arrival_city,fullness,revenue,change_flight_revenue_delta
247,136807,PG0480,SU9,Belgorod,70.103093,474333.0,-1.0
205,136642,PG0480,SU9,Belgorod,65.979381,475467.0,-1.0
255,136844,PG0480,SU9,Belgorod,81.443299,518433.0,-1.0


,flight_id,flight_no,aircraft_code,arrival_city,fullness,revenue,change_flight_revenue_delta
247,136807,PG0480,SU9,Belgorod,70.103093,474333.0,-1.0
205,136642,PG0480,SU9,Belgorod,65.979381,475467.0,-1.0
255,136844,PG0480,SU9,Belgorod,81.443299,518433.0,-1.0
267,136887,PG0480,SU9,Belgorod,80.412371,539667.0,-1.0
273,136922,PG0480,SU9,Belgorod,78.350515,551133.0,-1.0
201,136620,PG0480,SU9,Belgorod,81.443299,557967.0,-1.0
250,136823,PG0480,SU9,Belgorod,80.412371,562600.0,-1.0
277,136937,PG0480,SU9,Belgorod,83.505155,569433.0,-1.0
268,136888,PG0480,SU9,Belgorod,81.443299,569433.0,-1.0
215,136678,PG0480,SU9,Belgorod,82.474227,574533.0,-1.0


In [12]:
# посмотрим самые незаполненные рейсы
flights_top = df[
    (df['revenue'] > 0) &
    (df['fullness'] <= 75)
][['flight_id'] + columns].sort_values('fullness', ascending=True)

for x in list(flights_top['flight_id']):
    suggested_flights[x] = 1 if x not in suggested_flights else suggested_flights[x] + 1

suggested_flights_list = suggested_flights_list.append(flights_top[:3])
display(suggested_flights_list)
display(flights_top)


,flight_id,flight_no,aircraft_code,arrival_city,fullness,revenue,change_flight_revenue_delta
247,136807,PG0480,SU9,Belgorod,70.103093,474333.0,-1.0
205,136642,PG0480,SU9,Belgorod,65.979381,475467.0,-1.0
255,136844,PG0480,SU9,Belgorod,81.443299,518433.0,-1.0
205,136642,PG0480,SU9,Belgorod,65.979381,475467.0,-1.0
247,136807,PG0480,SU9,Belgorod,70.103093,474333.0,-1.0
146,136352,PG0252,733,Moscow,70.769231,1199067.0,60000.0


,flight_id,flight_no,aircraft_code,arrival_city,fullness,revenue,change_flight_revenue_delta
205,136642,PG0480,SU9,Belgorod,65.979381,475467.0,-1.0
247,136807,PG0480,SU9,Belgorod,70.103093,474333.0,-1.0
146,136352,PG0252,733,Moscow,70.769231,1199067.0,60000.0
120,136253,PG0252,733,Moscow,71.538462,1237933.0,58800.0
92,136122,PG0252,733,Moscow,74.615385,1257667.0,60000.0
147,136360,PG0252,733,Moscow,74.615385,1282067.0,60000.0
159,136418,PG0252,733,Moscow,74.615385,1234467.0,60000.0


In [13]:
# Посчитаем общее количество перевезённых пассажиров

directions = []

def get_direction(row):
    direction = row['departure_city'] + ' - ' + row['arrival_city']
    if direction not in directions:
        directions.append(direction)
    return direction

df['direction'] = df.apply(lambda x: get_direction(x), axis=1)

table_data = []

for d in directions:
    d_df = df[df['direction']==d]
    for t in d_df['seats_amount'].unique():
        t_df = d_df[d_df['seats_amount']==t]
        tickets = t_df['tickets_amount'].sum()
        seats = t_df['seats_amount'].sum()
        count = t_df['seats_amount'].count()
        result = round(tickets / t)
    if len(d_df['seats_amount'].unique()) > 1:
        print('Oops..')
    table_data.append([d, tickets, count, seats, result, count-result])

pd.DataFrame(table_data, columns=['Направление', 'Общее кол-во пассажиров', 'Всего рейсов', 'Вместительность рейсов', 'Достаточно рейсов', 'Можно отказаться'])

,Направление,Общее кол-во пассажиров,Всего рейсов,Вместительность рейсов,Достаточно рейсов,Можно отказаться
0,Moscow - Anapa,10408,90,11700,80,10
1,Anapa - Moscow,10210,90,11700,79,11
2,Anapa - Novokuznetsk,0,13,1690,0,13
3,Anapa - Belgorod,8141,90,8730,84,6
4,Novokuznetsk - Anapa,0,13,1690,0,13
5,Belgorod - Anapa,8327,90,8730,86,4
